1.Easy

Импортируем библиотеки, загрузим данные:

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('archive (4)/singapore_airlines_reviews.csv')
df.head()

,published_date,published_platform,rating,type,text,title,helpful_votes
0,2024-03-12T14:41:14-04:00,Desktop,3,review,We used this airline to go from Singapore to L...,Ok,0
1,2024-03-11T19:39:13-04:00,Desktop,5,review,The service on Singapore Airlines Suites Class...,The service in Suites Class makes one feel lik...,0
2,2024-03-11T12:20:23-04:00,Desktop,1,review,"Booked, paid and received email confirmation f...",Don’t give them your money,0
3,2024-03-11T07:12:27-04:00,Desktop,5,review,"Best airline in the world, seats, food, servic...",Best Airline in the World,0
4,2024-03-10T05:34:18-04:00,Desktop,2,review,Premium Economy Seating on Singapore Airlines ...,Premium Economy Seating on Singapore Airlines ...,0


Уберем пунктуацию:

In [3]:
def preprocess_text(text):
    punctuation = [".", ",", "!", "?", ":", ";", "-", "(", ")"]
    lst = text.split()
    fixed_text = []
    for word in lst:
        for Mark in punctuation:
            word = word.replace(Mark, "")
        fixed_text.append(word.lower())
    s = " ".join(fixed_text)
    return s

In [4]:
new_text = df["text"].apply(preprocess_text)
new_text

0       we used this airline to go from singapore to l...
1       the service on singapore airlines suites class...
2       booked paid and received email confirmation fo...
3       best airline in the world seats food service a...
4       premium economy seating on singapore airlines ...
                              ...                        
9995    first part done with singapore airlines  accep...
9996    and again a great flight with singapore air gr...
9997    we flew business class from frankfurt via sing...
9998    as always the a380 aircraft was spotlessly pre...
9999    as always singapore airlines has done it again...
Name: text, Length: 10000, dtype: object

Преобразуем в TF-IDF, обучим логистическую регрессию, будем использовать метрику f1:

In [5]:
warnings.filterwarnings("ignore", category=UserWarning) #чтобы не всплывал ConvergenceWarning
vectorizer = TfidfVectorizer()
new_train = vectorizer.fit_transform(new_text)
model = LogisticRegression()
x_train, x_test, y_train, y_test = train_test_split(new_train, df["rating"], train_size=0.8)
model.fit(x_train, y_train)
prediction = model.predict(x_test)
print(f1_score(prediction, y_test, average="weighted"))

0.7176930818076696


2.Normal

Удалим стоп-слова и применим лемматизацию:

In [6]:
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
def lemmatization_and_del_stopw(text):
    preprocessed_text = preprocess_text(text)
    lst = preprocessed_text.split()
    filtered_and_lemmatized_lst = [lemmatizer.lemmatize(word) for word in lst if word not in stop_words]
    filtered_text = " ".join(filtered_and_lemmatized_lst)
    return filtered_text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/banana_cat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
new_text = df["text"].apply(lemmatization_and_del_stopw)
vectorizer = TfidfVectorizer()
new_train = vectorizer.fit_transform(new_text)
x_train, x_test, y_train, y_test = train_test_split(new_train, df["rating"], train_size=0.8)

lrmodel = LogisticRegression()
lrmodel.fit(x_train, y_train)
prediction = lrmodel.predict(x_test)
print("LogisticRegression:", f1_score(prediction, y_test, average="weighted"))

tree_model = DecisionTreeClassifier()
tree_model.fit(x_train, y_train)
prediction = tree_model.predict(x_test)
print("DecisionTreeClassifier:", f1_score(prediction, y_test, average="weighted"))

svm_model = SVC(kernel = "rbf")
svm_model.fit(x_train, y_train)
prediction = svm_model.predict(x_test)
print("SVM(RBF):", f1_score(prediction, y_test, average="weighted"))

LogisticRegression: 0.7075165438769789
DecisionTreeClassifier: 0.5343393731090034
SVM(RBF): 0.7219223442218528


Как видно, лучший результат показала модель SVM на радиальном базисном ядре.

Придумаем отзывы, проверим работу модели:

In [8]:
good_review = '''The service was incredibly slow, and the staff were rude and inattentive. The food was cold and tasted stale.
Overall, it was a disappointing experience, and I would not recommend this restaurant to anyone. i will never ever use this fucking airlines
'''
bad_review  = '''The service at this restaurant was excellent. The staff were friendly and attentive. The food was delicious and beautifully presented.
Overall, it was a wonderful dining experience, and I would definitely recommend it to others.'''
reviews = [bad_review, good_review]
new_df = pd.DataFrame(reviews, columns=['reviews'])
text = new_df['reviews'].apply(lemmatization_and_del_stopw)
vectorizer = TfidfVectorizer()
new_test = vectorizer.fit_transform(new_text)
print(svm_model.predict(new_test[0]))
print(svm_model.predict(new_test[1]))

[3]
[5]
